In [28]:
%%writefile ../src/PracOrder.sol
pragma solidity ^0.6.0;

contract PracOrderEvent {
    uint unitPrice = 10;
    event OrderLog(address indexed _from, bytes2 _itemId, uint indexed quantity, string shipping);
    
    address payable owner;
    constructor() public {
        owner = msg.sender;
    }


    function order(bytes2 _itemId, uint quantity, string memory shipping) public payable {
        uint256 orderAmount = quantity * unitPrice;
        require(msg.value == orderAmount);
        emit OrderLog(msg.sender, _itemId, quantity, shipping);
    }
    
    
    function getBalance() view public returns(uint) {
        return address(this).balance;
    }
}

Writing ../src/PracOrder.sol


In [29]:
!solc ../src/PracOrder.sol --combined-json abi > ..\src\PracOrderABI.json

In [31]:
!solc ../src/PracOrder.sol --combined-json bin > ..\src\PracOrderBIN.json

In [34]:
%%writefile ../src/PracOrderDeploy.js
var Web3=require('web3');
var _abiJson = require('./PracOrderABI.json');
var _binJson = require('./PracOrderBIN.json');

var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));


contractNames=Object.keys(_abiJson.contracts); 
contractName=contractNames[0];
console.log("- contract name: ", contractName);

_abiArray=JSON.parse(_abiJson.contracts[contractName].abi);  
_bin=_binJson.contracts[contractName].bin;

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 1000000}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()

Overwriting ../src/PracOrderDeploy.js


In [35]:
!node ../src/PracOrderDeploy.js

- contract name:  ../src/PracOrder.sol:PracOrderEvent
Deploying the contract from 0xC5F7533Aeaaa59A63eB800B97aBa1EF4DE08A9B6
hash: 0xc4193baffa29bdb22dc13b124b3d011a451c75fe8e3ac6e222d936549e5583e7
---> The contract deployed to: 0x221d4dfc48ee26e0AC113C64fd5B79b7cF67BdBD


In [46]:
%%writefile ../src/PracOrderUser.js
var Web3=require('web3');
var _abiJson = require('./PracOrderABI.json');


var web3;

//web3 = new Web3(new Web3.providers.WebsocketProvider("ws://localhost:8345"));
web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));


contractNames=Object.keys(_abiJson.contracts); 
contractName=contractNames[0];
console.log("- contract name: ", contractName); //or console.log(contractName);
_abiArray=JSON.parse(_abiJson.contracts[contractName].abi);    //JSON parsing needed!!

 var _instance = new web3.eth.Contract(_abiArray, "0x221d4dfc48ee26e0AC113C64fd5B79b7cF67BdBD");

var event = _instance.events.OrderLog({fromBlock: 0}, function (error, result) {
    if (!error) {
        console.log("Event fired: " + JSON.stringify(result.returnValues));
    }
});

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
   
    my = await _instance.methods.order("0x1234",  3, "20 2-gil Hongji-dong Jongro-gu Seoul")
        .send({from: accounts[0], gas: 100000, value:30})

}

doIt()

Overwriting ../src/PracOrderUser.js


In [47]:
!node ../src/PracOrderUser.js

- contract name:  ../src/PracOrder.sol:PracOrderEvent
Account: 0xC5F7533Aeaaa59A63eB800B97aBa1EF4DE08A9B6
Balance before: 99978859605999999757
